In [11]:
import os
import h5py
import pathlib
import logging
import numpy as np
import pandas as pd

import tensorflow
if int(tensorflow.__version__.split(".")[0]) >= 2:
    import tensorflow.compat.v1 as tf
else:
    import tensorflow as tf

from datetime import datetime
import multiprocessing
import argparse
import json
import pathlib
# from tqdm import tqdm
import sys
# sys.path.append(pathlib.Path(__file__).parent.absolute())
from commons import input_iterator

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
from commons import cnn_bi_lstm_model

os.chdir('/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/')
import pickle
from torch import nn

## Custom trained model

In [12]:
tf.reset_default_graph()
p = max(1, multiprocessing.cpu_count()//2)
sess = tf.Session(config=tf.ConfigProto(inter_op_parallelism_threads=p, intra_op_parallelism_threads=p))
tf.saved_model.loader.load(sess, ["serve"], "/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/custom_weights/CUSTOM_MODEL")

INFO:tensorflow:Restoring parameters from /home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/custom_weights/CUSTOM_MODEL/variables/variables


INFO:tensorflow:Restoring parameters from /home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/custom_weights/CUSTOM_MODEL/variables/variables


meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_BFLOAT16
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "AddN"
      input_arg {
        name: "inputs"
        type_attr: "T"
        number_attr: "N"
      }
      output_arg {
        name: "sum"
        type_attr: "T"
      }
      attr {
        name: "N"
        type: "int"
        has_minimum: t

In [4]:
vars = sess.graph.get_collection('trainable_variables')
weights = {}
for v in vars:
    weights[v.name] = sess.run(v)  # retrieve the value from the tf backend
weights_list = [(k, v) for k, v in weights.items()]
for ele in weights_list:
    print(ele[0], ele[1].shape)

model/conv2d/kernel:0 (5, 3, 1, 64)
model/conv2d/bias:0 (64,)
model/conv2d_1/kernel:0 (5, 1, 64, 128)
model/conv2d_1/bias:0 (128,)
model/conv2d_2/kernel:0 (5, 1, 128, 256)
model/conv2d_2/bias:0 (256,)
model/conv2d_3/kernel:0 (5, 1, 256, 512)
model/conv2d_3/bias:0 (512,)
model/conv2d_4/kernel:0 (5, 1, 512, 512)
model/conv2d_4/bias:0 (512,)
model/dense/kernel:0 (6144, 512)
model/dense/bias:0 (512,)
bidirectional_rnn/fw/lstm_cell/kernel:0 (640, 512)
bidirectional_rnn/fw/lstm_cell/bias:0 (512,)
bidirectional_rnn/bw/lstm_cell/kernel:0 (640, 512)
bidirectional_rnn/bw/lstm_cell/bias:0 (512,)
dense/kernel:0 (256, 2)
dense/bias:0 (2,)


In [5]:
sess.graph.get_tensor_by_name("output:0")

<tf.Tensor 'output:0' shape=(?,) dtype=int64>

## Pre-trained CHAP ADULT

In [28]:
tf.reset_default_graph()
p = max(1, multiprocessing.cpu_count()//2)
sess = tf.Session(config=tf.ConfigProto(inter_op_parallelism_threads=p, intra_op_parallelism_threads=p))
tf.saved_model.loader.load(sess, ["serve"], "/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/pre-trained-models/CHAP_ALL_ADULTS")

INFO:tensorflow:Restoring parameters from /home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/pre-trained-models/CHAP_ALL_ADULTS/variables/variables


INFO:tensorflow:Restoring parameters from /home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/pre-trained-models/CHAP_ALL_ADULTS/variables/variables


meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_BFLOAT16
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "Assign"
      input_arg {
        name: "ref"
        type_attr: "T"
        is_ref: true
      }
      input_arg {
        name: "value"
        type_attr: "T"
      }
      output_arg {
        name: "output_ref"
        type_attr: "T"
        

In [23]:
saver = tf.train.Saver()
saver.save(sess, 'my-model.ckpt')

'my-model.ckpt'

In [30]:
vars = sess.graph.get_collection('trainable_variables')
weights = {}
for v in vars:
    weights[v.name] = sess.run(v)  # retrieve the value from the tf backend
weights_list = [(k, v) for k, v in weights.items()]
for ele in weights_list:
    print(ele[0], ele[1].shape)

model/conv2d/kernel:0 (5, 3, 1, 64)
model/conv2d/bias:0 (64,)
model/conv2d_1/kernel:0 (5, 1, 64, 128)
model/conv2d_1/bias:0 (128,)
model/conv2d_2/kernel:0 (5, 1, 128, 256)
model/conv2d_2/bias:0 (256,)
model/conv2d_3/kernel:0 (5, 1, 256, 512)
model/conv2d_3/bias:0 (512,)
model/conv2d_4/kernel:0 (5, 1, 512, 512)
model/conv2d_4/bias:0 (512,)
model/dense/kernel:0 (6144, 512)
model/dense/bias:0 (512,)
bidirectional_rnn/fw/lstm_cell/kernel:0 (640, 512)
bidirectional_rnn/fw/lstm_cell/bias:0 (512,)
bidirectional_rnn/bw/lstm_cell/kernel:0 (640, 512)
bidirectional_rnn/bw/lstm_cell/bias:0 (512,)
dense/kernel:0 (256, 1)
dense/bias:0 (1,)


In [31]:
sess.graph.get_tensor_by_name("output:0")

<tf.Tensor 'output:0' shape=(?, 42) dtype=float32>

In [26]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    new_saver = tf.train.import_meta_graph("/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/checkpoint_new/my-model.ckpt.meta")
    new_saver.restore(sess, tf.train.latest_checkpoint('/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/checkpoint_new'))
    vars = sess.graph.get_collection('trainable_variables')
    weights = {}
    for v in vars:
        weights[v.name] = sess.run(v)  # retrieve the value from the tf backend
    weights_list = [(k, v) for k, v in weights.items()]
    for ele in weights_list:
        print(ele[0], ele[1].shape)

INFO:tensorflow:Restoring parameters from /home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/checkpoint_new/my-model.ckpt


INFO:tensorflow:Restoring parameters from /home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/checkpoint_new/my-model.ckpt


model/conv2d/kernel:0 (5, 3, 1, 64)
model/conv2d/bias:0 (64,)
model/conv2d_1/kernel:0 (5, 1, 64, 128)
model/conv2d_1/bias:0 (128,)
model/conv2d_2/kernel:0 (5, 1, 128, 256)
model/conv2d_2/bias:0 (256,)
model/conv2d_3/kernel:0 (5, 1, 256, 512)
model/conv2d_3/bias:0 (512,)
model/conv2d_4/kernel:0 (5, 1, 512, 512)
model/conv2d_4/bias:0 (512,)
model/dense/kernel:0 (6144, 512)
model/dense/bias:0 (512,)
bidirectional_rnn/fw/lstm_cell/kernel:0 (640, 512)
bidirectional_rnn/fw/lstm_cell/bias:0 (512,)
bidirectional_rnn/bw/lstm_cell/kernel:0 (640, 512)
bidirectional_rnn/bw/lstm_cell/bias:0 (512,)
dense/kernel:0 (256, 1)
dense/bias:0 (1,)


In [19]:
vars = sess.graph.get_collection('trainable_variables')
weights = {}
for v in vars:
    weights[v.name] = sess.run(v)  # retrieve the value from the tf backend
weights_list = [(k, v) for k, v in weights.items()]
for ele in weights_list:
    print(ele[0], ele[1].shape)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/home/animeshkumar/anaconda3/envs/deep_postures/lib/python3.7/site-packages/tensorflow_core/python/client/session.py", line 1365, in _do_call
  File "/home/animeshkumar/anaconda3/envs/deep_postures/lib/python3.7/site-packages/tensorflow_core/python/client/session.py", line 1350, in _run_fn
  File "/home/animeshkumar/anaconda3/envs/deep_postures/lib/python3.7/site-packages/tensorflow_core/python/client/session.py", line 1443, in _call_tf_sessionrun
tensorflow.python.framework.errors_impl.FailedPreconditionError: Attempting to use uninitialized value model/conv2d/kernel
	 [[{{node _retval_model/conv2d/kernel_0_0}}]]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/animeshkumar/anaconda3/envs/deep_postures/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_638871/3428